# Analysis of Standardized Testing in Schools Using Pandas

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# rename columns
school_data = school_data.rename(columns={'school_name':'School Name','type':'School Type','size':'Total Students','budget':'Total School Budget'})
student_data = student_data.rename(columns={'school_name':'School Name'})

# print column headers
print(school_data.columns)
print(student_data.columns)

Index(['School ID', 'School Name', 'School Type', 'Total Students',
       'Total School Budget'],
      dtype='object')
Index(['Student ID', 'student_name', 'gender', 'grade', 'School Name',
       'reading_score', 'math_score'],
      dtype='object')


In [2]:
# calculate Per Student Budget
school_data['Per Student Budget'] = school_data['Total School Budget'] / school_data['Total Students']

# assign Per Student Budget values to bins
spend_bins = [0,585,630,645,680]
spend_labels = ['<=$585','$586-$630','$631-$645','$646-$680']
school_data['Spending Ranges (Per Student)'] = pd.cut(school_data['Per Student Budget'], bins=spend_bins, labels=spend_labels)

# assign size of school to bins
size_bins = [0,1000,2000,5000]
size_labels = ['Small (<=1000)', 'Medium (1001-2000)', 'Large (2001-5000)']
school_data['School Size'] = pd.cut(school_data['Total Students'], bins=size_bins, labels=size_labels)

# apply formatting
pd.options.display.float_format = '{:,.2f}'.format
school_data['Per Student Budget'] = school_data['Per Student Budget'].map('${:,.2f}'.format)

# preview table
school_data.head()

,School ID,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Spending Ranges (Per Student),School Size
0,0,Huang High School,District,2917,1910635,$655.00,$646-$680,Large (2001-5000)
1,1,Figueroa High School,District,2949,1884411,$639.00,$631-$645,Large (2001-5000)
2,2,Shelton High School,Charter,1761,1056600,$600.00,$586-$630,Medium (1001-2000)
3,3,Hernandez High School,District,4635,3022020,$652.00,$646-$680,Large (2001-5000)
4,4,Griffin High School,Charter,1468,917500,$625.00,$586-$630,Medium (1001-2000)


In [3]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data[['School Name','grade','math_score','reading_score']], school_data[['School Name','School Type','Per Student Budget','Spending Ranges (Per Student)','School Size']], on='School Name',how='outer')

# Rename columns
school_data_complete = school_data_complete.rename(columns={'school_name':'School Name'})

# preview table
school_data_complete.head()

,School Name,grade,math_score,reading_score,School Type,Per Student Budget,Spending Ranges (Per Student),School Size
0,Huang High School,9th,79,66,District,$655.00,$646-$680,Large (2001-5000)
1,Huang High School,12th,61,94,District,$655.00,$646-$680,Large (2001-5000)
2,Huang High School,12th,60,90,District,$655.00,$646-$680,Large (2001-5000)
3,Huang High School,12th,58,67,District,$655.00,$646-$680,Large (2001-5000)
4,Huang High School,9th,84,97,District,$655.00,$646-$680,Large (2001-5000)


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [4]:
# calculate total students
total_students = len(student_data['Student ID'].unique())
math_df = student_data.loc[student_data['math_score'] >= 70]
reading_df = student_data.loc[student_data['reading_score'] >= 70]
overall_df = student_data.loc[(student_data['math_score'] >= 70) & (student_data['reading_score'] >= 70)]

# calculate district metrics and store in dataframe
district_summary_df = pd.DataFrame({
  'Total Schools': [len(school_data['School Name'].unique())],
  'Total Students': total_students,
  'Total Budget': sum(school_data['Total School Budget']),
  'Average Math Score': student_data['math_score'].mean(),
  'Average Reading Score': student_data['reading_score'].mean(),
  '% Passing Math': len(math_df) / total_students * 100,
  '% Passing Reading': len(reading_df) / total_students * 100,
  '% Overall Passing': len(overall_df) / total_students * 100
})

# apply formatting
district_summary_df['Total Students'] = district_summary_df['Total Students'].map('{:,.0f}'.format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map('${:,.2f}'.format)

# show results
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [5]:
# dependencies
from functools import reduce

# calculate average math and reading scores per school
school_average_df = school_data_complete.groupby('School Name')[['math_score','reading_score']].mean()
school_average_df = school_average_df.rename(columns={'math_score':'Average Math Score','reading_score':'Average Reading Score'})

# count passing students in math, reading, and overall per school
math_school_df = math_df.groupby('School Name')['Student ID'].count()
reading_school_df = reading_df.groupby('School Name')['Student ID'].count()
overall_school_df = overall_df.groupby('School Name')['Student ID'].count()

# merge multiple tables sequentially by School Name
dfs_to_merge = [school_data[['School Name', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget']], 
                school_average_df, 
                math_school_df, 
                reading_school_df, 
                overall_school_df]
merged_df = reduce(lambda left,right: pd.merge(left,right, on=['School Name'], how='outer'), dfs_to_merge)

# calculate percent of students passing math, reading, and overall per school
merged_df['% Passing Math'] = merged_df['Student ID_x'] / merged_df['Total Students'] * 100
merged_df['% Passing Reading'] = merged_df['Student ID_y'] / merged_df['Total Students'] * 100
merged_df['% Overall Passing'] = merged_df['Student ID'] / merged_df['Total Students'] * 100

# sort and select columns for final summary table of results by school
school_summary_df = merged_df.sort_values('School Name', ascending=True).set_index('School Name', drop=True)
school_summary_df = school_summary_df[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]

# apply formatting
school_summary_df['Total Students'] = school_summary_df['Total Students'].map('{:,.0f}'.format)
school_summary_df['Total School Budget'] = school_summary_df['Total School Budget'].map('${:,.2f}'.format)

# preview data
school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [6]:
# sort for top 5 schools by overall passing percentage value
top_schools_df = school_summary_df.sort_values('% Overall Passing', ascending=False)
top_schools_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [7]:
# sort for bottom 5 schools by overall passing percentage value
bottom_schools_df = school_summary_df.sort_values('% Overall Passing', ascending=True)
bottom_schools_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
# calculate average math scores by school and grade and save to dataframe
math_school_df = pd.DataFrame(school_data_complete.groupby(['School Name','grade'])['math_score'].mean())

# format dataframe from long to wide format and reorder columns
wide_math = math_school_df.math_score.unstack()
wide_math.columns.name = None
wide_math = wide_math[['9th','10th','11th','12th']]

# show results
wide_math

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [9]:
# calculate average reading scores by school and grade and save to dataframe
reading_school_df = pd.DataFrame(school_data_complete.groupby(['School Name','grade'])['reading_score'].mean())

# format dataframe from long to wide format and reorder columns
wide_reading = reading_school_df.reading_score.unstack()
wide_reading.columns.name = None
wide_reading = wide_reading[['9th','10th','11th','12th']]

# show results
wide_reading

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
# join school summary data with fields for grouping
school_extended_df = pd.merge(school_summary_df[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']],school_data[['School Name', 'Spending Ranges (Per Student)', 'School Size', 'School Type']],on='School Name',how='inner')

# calculate averages by spending range category based on the averages and percentages calculated per school
# note: the average and percent values vary slightly from the results below if the raw numbers (grades per student per spending category) are used in calculations instead
scores_by_spending_df = school_extended_df.groupby('Spending Ranges (Per Student)').mean()
scores_by_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<=$585,83.46,83.93,93.46,96.61,90.37
$586-$630,81.90,83.16,87.13,92.72,81.42
$631-$645,78.52,81.62,73.48,84.39,62.86
$646-$680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [11]:
# calculate averages by school size category based on the averages and percentages calculated per school
# note: the average and percent values vary slightly from the results below if the raw numbers (grades per student per school size category) are used in calculations instead
scores_by_size_df = school_extended_df.groupby('School Size').mean()
scores_by_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<=1000),83.82,83.93,93.55,96.10,89.88
Medium (1001-2000),83.37,83.86,93.60,96.79,90.62
Large (2001-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [12]:
# calculate averages by school type based on the averages and percentages calculated per school
# note: the average and percent values vary slightly from the results below if the raw numbers (grades per student per school type) are used in calculations instead
scores_by_type_df = school_extended_df.groupby('School Type').mean()
scores_by_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
